In [1]:
from core.database.supabase_client import get_supabase_client
from yt_rag.llm_service.gemini_client import get_gemini_client


llm_client = get_gemini_client()
supabase = get_supabase_client()

In [26]:
# tools

# Multimodal RAG tool 
import json
import requests
from yt_rag.vector_store.embeddings import create_text_embeddings
from yt_rag.vector_store.pg_vector_operations import PG_Vector_search

def get_relevent_multimodal_data(video_id: str, query: str):
    query_embedding = create_text_embeddings(query)
    vector_search = PG_Vector_search()
    
    frames = vector_search.match_frames(video_id, query_embedding)
    transcripts = vector_search.match_transcript(video_id, query_embedding)
    print(frames, transcripts)
    as_messages = {
        "role": "user",
        "parts": [
            {"text": "These are the relevant multimodal (Contains youtube frames and text transcript chunks) data to user's question. Use this data to answer intelligently if helpfull"}
        ]
    }
    for frame in frames:
        data = requests.get(frame["frame_url"][1:])
        image_bytes = data.content
        as_messages["parts"].append(
            {"inline_data":{ "mime_type":"image/jpeg", "data": image_bytes }}
        )
    data = ""
    for transcript in transcripts:
        json_data = json.loads(transcript["content"])
        data += json_data["page_content"]
    as_messages["parts"].append(
        {"text": data}
    )
    
    return as_messages
        
    
# Websearch tool


In [ ]:
from google.genai.types import Tool
rag_tool = Tool(
    function_declarations=[rag_function_declaration]
)

In [ ]:
from google.genai.types import GenerateContentConfig, GoogleSearch, Tool

web_search_tool = Tool(google_search=GoogleSearch())

config = GenerateContentConfig(
    tools=[web_search_tool, ]
)

response = llm_client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Latest news about tesla? But don't use any websearch capabilities",
    config=config
)

In [27]:
from google.genai import types
from google.genai.types import GoogleSearch, Tool
rag_function_declaration = types.FunctionDeclaration(
    name="get_relevent_multimodal_data",
    description="get relevent frames and transcript chunks to users query from youtube video",
    parameters=types.Schema(
        type=types.Type.OBJECT,
        properties={
            "video_id": types.Schema(type=types.Type.STRING, description="Unique identifier of a YouTube video"),
            "query": types.Schema(type=types.Type.STRING, description="Efficient query for the RAG tool to find relevant data"),
        },
        required=["video_id", "query"]
    ),
)

rag_tool = Tool(
    function_declarations=[rag_function_declaration]
)
web_search_tool = Tool(
    google_search=GoogleSearch()
)
tools=[
    rag_tool
]

In [105]:
response = llm_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[{
        "role":"user",
        "parts":[{"text": "What is jorem powell talking in this video? link to video=https://m.youtube.com/watch?v=wCNIhFdhgLE"}]
    }],
    config=types.GenerateContentConfig(
        temperature=0.75,
        system_instruction= "You are a youtube assistant assistant. You have access to multimodal rag tool \"get_relevent_multimodal_data\" which u can use to relevant frames and text chunks of the video. To use the tool u have to send video_id and query to the tool. video_id is the unique indentifier to a youtube video for example \"jU4d39Z_9GU\" is video_id for youtube url \"https://m.youtube.com/watch?v=jU4d39Z_9GU\"",
        tools= tools
    )
)

In [107]:
response.function_calls

[FunctionCall(
   args={
     'query': 'jorem powell talking',
     'video_id': 'wCNIhFdhgLE'
   },
   name='get_relevent_multimodal_data'
 )]

In [28]:
system_instruction = """You are a youtube assistant assistant. You have access to multimodal rag tool 
\"get_relevent_multimodal_data\" which u can use to relevant frames and text chunks of the video.
To use the tool u have to send video_id and query to the tool. video_id is the unique indentifier 
to a youtube video for example \"jU4d39Z_9GU\" is video_id for youtube url \"https://m.youtube.com/watch?v=jU4d39Z_9GU\"","""

In [54]:
# Define the agent
from google.genai.types import Part
tools_dict = {
    "get_relevent_multimodal_data": get_relevent_multimodal_data
}

class LLMClient:
    def __init__(self, video_id: str, user_id: str, chat_id: str):
        self.history = []
        self.llm_client = get_gemini_client()
        self.supabase = get_supabase_client()
        self.video_id = video_id
        self.user_id = user_id
        self.chat_id = chat_id
        self.llm_tools = tools
        # print(self.llm_tools)
        
    def load_history(self):
        response = (
            supabase.table("chat")
            .select("chat_messages(role, content)")
            .eq("video_id", self.video_id)
            .eq("user_id", self.user_id)
            .order("created_at", desc=False, foreign_table="chat_messages")
            .execute()
        )
        for message in response.data[0]["chat_messages"]:
            if message["role"] == "USER":
                self.history.append({
                    "role": "user",
                    "parts": [{
                        "text": message["content"]
                    }]
                })
            else:
                self.history.append({
                    "role": "model",
                    "parts": [{
                        "text": message["content"]
                    }]
                })
        print(f"History loaded successfully from supabase")
        
    def run(self, prompt):
        
        self.history.append({
            "role": "user",
            "parts": [{
                "text":prompt
            }]
        })
        
        contents = self.history
        
        insert_user_messages = (
            supabase.table("chat_messages").insert({
                "chat_id": self.chat_id,
                "role": "USER",
                "content": prompt
            }).execute())
        print("Success: Insert user message into supabase")
        
        while True:
            print(contents)
            
            response = llm_client.models.generate_content(
                model="gemini-2.0-flash",
                contents = contents,
                config = {
                    "system_instruction": system_instruction,
                    "temperature": 0.5,
                    "tools": self.llm_tools
                },
            )
            print("Success: Response generated by model")
            
            if response.function_calls:
                print(f"Model requesed to tool call: {response.function_calls}")
                requested_tool_calls = [{"tool_name": function_call.name, "tool_args": function_call.args} for function_call in response.function_calls]
                print(f"Formed tool calls: {requested_tool_calls}")
                self.supabase.table("chat_messages").insert({
                    "chat_id": self.chat_id,
                    "role": "ASSISTANT",
                    "content": "check check",
                    "tool_calls": requested_tool_calls
                }).execute()
                print("Model requested tool call")
                tool_results = []
                for function_call in response.function_calls:
                    tool_function = tools_dict["get_relevent_multimodal_data"]
                    relevant_content = tool_function(
                        video_id= function_call.args["video_id"],
                        query= function_call.args["query"]
                    )
                    contents.append(relevant_content)
                print("Success: Tools completed execution")
            else:
                self.supabase.table("chat_messages").insert({
                    "chat_id":self.chat_id,
                    "role":"ASSISTANT",
                    "content":response.text
                }).execute()
                contents.append({
                    "role":"user",
                    "parts":[{
                        "text":response.text
                    }]
                })
                print(f"Success: Model generated final response=> {response.text}")
                return response.text


In [55]:
llm = LLMClient(
    video_id="wbX0zOdCpNM",
    user_id= "c2c23e89-3a2f-4da6-9035-af29b4da31b2",
    chat_id="09429eb7-0d94-4f9c-98fe-9ae5f88b4d0d",
)

llm.load_history()

while True:
    query = input("Enter ur query (No to quit): ")
    if query == "q":
        print("Chat is ended...")
        break
    else:
        response = llm.run(query)

History loaded successfully from supabase
Success: Insert user message into supabase
[{'role': 'user', 'parts': [{'text': 'what else he is talking about ott market share in india'}]}]
Success: Response generated by model
Success: Model generated final response=> Could you please provide the video ID so I can look into the OTT market share information?

Success: Insert user message into supabase
[{'role': 'user', 'parts': [{'text': 'what else he is talking about ott market share in india'}]}, {'role': 'user', 'parts': [{'text': 'Could you please provide the video ID so I can look into the OTT market share information?\n'}]}, {'role': 'user', 'parts': [{'text': 'https://www.youtube.com/watch?v=wbX0zOdCpNM'}]}]
Success: Response generated by model
Model requesed to tool call: [FunctionCall(
  args={
    'query': 'OTT market share in India',
    'video_id': 'wbX0zOdCpNM'
  },
  name='get_relevent_multimodal_data'
)]
Formed tool calls: [{'tool_name': 'get_relevent_multimodal_data', 'tool_ar

In [42]:
response = supabase.table("chat_messages").update({"tool_calls":[{'tool_name': 'get_relevent_multimodal_data', 'tool_args': {'query': 'interest rates and images', 'video_id': 'wCNIhFdhgLE'}}]}).eq("id", "74bfea0b-b4d9-4288-93f4-49222ee68ae9").execute()

In [43]:
response

APIResponse[~_ReturnT](data=[{'id': '74bfea0b-b4d9-4288-93f4-49222ee68ae9', 'created_at': '2025-09-20T14:06:44.457088+00:00', 'chat_id': 'badaefd3-0a56-4d44-8781-a900f7555b21', 'content': 'Here\'s a breakdown of what Jerome Powell discusses regarding interest rates in the provided content:\n\n**Key Points:**\n\n*   **Interest Rate Cut:** The Federal Open Market Committee decided to lower the policy interest rate by a quarter percentage point.\n*   **Risk Management:** Powell refers to the interest rate cut as a "risk management cut."\n*   **Labor Market:** The labor market is described as being in very solid condition with strong job creation.\n*   **Inflation:** They expect inflation to move up, but maybe not as high as previously anticipated. The impact of tariffs on inflation has been slower and smaller than expected.\n\n**Image Description:**\n\nThe images show Jerome Powell, the Federal Reserve Chair, speaking at a podium. He is wearing a suit and tie. Behind him are flags, and th

In [ ]:

initial_messages = [
    {
        "role": "USER",
        "content": "What is this video about",
        "chat_id": "09429eb7-0d94-4f9c-98fe-9ae5f88b4d0d",
        "input_tokens": 40,
        "output_tokens": 80
    },
    {
        "role": "ASSISTANT",
        "content": "This video talks about football",
        "chat_id": "09429eb7-0d94-4f9c-98fe-9ae5f88b4d0d",
        "input_tokens": 50,
        "output_tokens": 100
    },
    {
        "role": "USER",
        "content": "How is football different from soccer",
        "chat_id": "09429eb7-0d94-4f9c-98fe-9ae5f88b4d0d",
        "input_tokens": 84,
        "output_tokens": 100
    },
    {
        "role": "ASSISTANT",
        "content": "LOL in america they call it that way both are same",
        "chat_id": "09429eb7-0d94-4f9c-98fe-9ae5f88b4d0d",
        "input_tokens": 122,
        "output_tokens": 232
    }
]
supabase.table("chat_messages").insert(initial_messages).execute()

In [55]:
response = (supabase.table("chat")
.select("chat_messages(role, content)")
.eq("video_id", "wCNIhFdhgLE")
.eq("user_id", "479681c0-8533-4e69-ad47-7847eb3c300a")
.order("created_at", desc=False, foreign_table="chat_messages")
.execute())

In [56]:
response.data[0]["chat_messages"]

[{'role': 'USER', 'content': 'What is special about janapese cuisine'},
 {'role': 'ASSISTANT',
  'content': "Japanese cuisine is very popular and there is so much to try. If it's your first time then don't forget sushi"}]

In [60]:
tools_dict = {
    "get_relevent_multimodal_data": get_relevent_multimodal_data
}

In [18]:
llm = LLMClient(
    video_id="wCNIhFdhgLE",
    user_id= "c2c23e89-3a2f-4da6-9035-af29b4da31b2",
    chat_id="09429eb7-0d94-4f9c-98fe-9ae5f88b4d0d",
)

llm.load_history()

while True:
    query = input("Enter ur query (No to quit): ")
    if query == "q":
        print("Chat is ended...")
        break
    else:
        response = llm.run(query)

History loaded successfully from supabase
Success: Insert user message into supabase
[{'role': 'user', 'parts': {'text': 'What is this video about'}}, {'role': 'model', 'parts': {'text': 'This video talks about football'}}, {'role': 'user', 'parts': {'text': 'How is football different from soccer'}}, {'role': 'model', 'parts': {'text': 'LOL in america they call it that way both are same'}}, {'role': 'user', 'parts': {'text': 'I am yogesh'}}]


ValidationError: 75 validation errors for _GenerateContentParameters
contents.Content
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=[{'role': 'user', 'parts'...'text': 'I am yogesh'}}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_attributes_type
contents.str
  Input should be a valid string [type=string_type, input_value=[{'role': 'user', 'parts'...'text': 'I am yogesh'}}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
contents.is-instance[Image]
  Input should be an instance of Image [type=is_instance_of, input_value=[{'role': 'user', 'parts'...'text': 'I am yogesh'}}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/is_instance_of
contents.File
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=[{'role': 'user', 'parts'...'te

In [16]:
contents

NameError: name 'contents' is not defined